In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Install required libraries
!pip install transformers torch vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.6 MB/s eta 0:00:0000:01


In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize DialoGPT
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Initialize chat history
chat_history_ids = torch.tensor([]).to(torch.int64)  # Empty tensor for initial chat history

def analyze_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)
    return sentiment_score

def chat_with_bot(input_text, sentiment):
    global chat_history_ids

    # Customize response based on sentiment
    if sentiment['compound'] >= 0.05:
        response_prefix = "That's great to hear! "
    elif sentiment['compound'] <= -0.05:
        response_prefix = "I'm sorry to hear that. "
    else:
        response_prefix = "Thanks for sharing. "

    # Tokenize input text and append end-of-sequence token
    new_user_input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')
    
    # Create attention mask
    attention_mask = torch.ones(new_user_input_ids.shape, dtype=torch.long)
    
    # Concatenate new input with previous chat history
    if chat_history_ids.size(0) > 0:
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
        attention_mask = torch.cat([torch.ones(chat_history_ids.shape, dtype=torch.long), attention_mask], dim=-1)
    else:
        bot_input_ids = new_user_input_ids
    
    # Generate response
    chat_history_ids = model.generate(
        bot_input_ids,
        attention_mask=attention_mask,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Extract and decode response
    bot_output = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    # Update chat history
    chat_history_ids = chat_history_ids[:, bot_input_ids.shape[-1]:]
    
    # Prepend sentiment-based response prefix
    bot_output = response_prefix + bot_output
    
    return bot_output

# Example usage
user_input = "I'm feeling really stressed today."
sentiment = analyze_sentiment(user_input)
response = chat_with_bot(user_input, sentiment)

print("Sentiment Analysis:", sentiment)
print("Chatbot Response:", response)


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Chatbot Response: I'm sorry to hear that. I'm feeling really stressed today. I'm sorry to hear that. I'm feeling really stressed today.
